In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import numpy as np
import tensorflow as tf
path = r'C:\\Users\\weso\\Desktop\\glove.6B\\'
id_dict = {}
word_dict = {}

matrix = []
dimensions = 300
with open(path + "glove.6B.300d.txt", 'r' , encoding="utf-8")  as f:
    lines = f.readlines()
    vocab_size = len(lines)
    
    matrix = np.zeros((vocab_size,dimensions),dtype=float)
    for line in lines:
        values = line.split()
        word = values[0].strip()
        id = len(id_dict)
        id_dict[word]=id
        word_dict[id] = word
        vector = np.asarray(values[1:], "double")
        matrix[id_dict[word],:] = vector
print(len(id_dict))

400000


In [3]:
matrix_normalized = tf.nn.l2_normalize(matrix,axis = 1)

In [5]:
pip install word2number

  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5580 sha256=ab6e4c3d2f7fb0ef47d8a0bed7ad1a423b4e47b03af28b045d58284a928490d3
  Stored in directory: c:\users\weso\appdata\local\pip\cache\wheels\cb\f3\5a\d88198fdeb46781ddd7e7f2653061af83e7adb2a076d8886d6
Successfully built word2number
Note: you may need to restart the kernel to use updated packages.


In [60]:
from word2number import w2n
numbers = {}
for word in id_dict:
    if word.startswith('-'):
        continue
    try:
        x = int(word)
        numbers[word] = ''
        pass
    except:
        pass
    if '-' in word:
        split = word.split('-')
        try:
            res = w2n.word_to_num(word)
        except:
            continue
        
        for i in range(len(split)):
            if split[i] in ('block','grade','point','on','hi-five','tier','hi','top','x'):
                isNumber = False
                break
            isNumber = True
            try:
                res = w2n.word_to_num(split[i])
                #x = int(res)
            except:
                isNumber = False
        if isNumber:
            numbers[str(word)] = ''
    else:
        try:
            res = w2n.word_to_num(word)
            numbers[str(word)] = ''
        except:
            pass
print(len(numbers))
print(numbers)

3665
{'one': '', 'two': '', 'three': '', 'million': '', 'four': '', 'billion': '', 'five': '', '1': '', '10': '', 'six': '', '2': '', '3': '', '20': '', 'point': '', '15': '', '30': '', '4': '', '11': '', '12': '', '5': '', 'seven': '', 'eight': '', '18': '', '6': '', '100': '', '25': '', '2008': '', '2006': '', '7': '', '2007': '', '8': '', '2010': '', '14': '', '2000': '', '13': '', '50': '', '16': '', '2009': '', '2004': '', '2005': '', 'nine': '', '17': '', '2001': '', '2003': '', '9': '', '24': '', '40': '', '2002': '', '2011': '', '1998': '', '19': '', '1996': '', '2012': '', '1999': '', '22': '', '21': '', '1994': '', '1997': '', '1995': '', '23': '', '26': '', '28': '', '27': '', '60': '', '31': '', '1993': '', '1992': '', '29': '', '200': '', '2013': '', '1991': '', '0': '', '500': '', '1990': '', '70': '', '80': '', '35': '', 'ten': '', '1989': '', '45': '', '300': '', '90': '', '65': '', '1988': '', '32': '', '1986': '', '400': '', '1987': '', '1984': '', '1980': '', '1985':

In [ ]:
min_tuple = (0,0)
min_value = 1

i = 0
number_list = list(numbers.keys())
for idA,numA in enumerate(number_list ):
    if i % 1 == 0:
        print(i)
    i += 1
    for numB in number_list[idA:]:
        if(numA == numB):
            continue
        id_a = id_dict[numA]
        id_b = id_dict[numB]
        weight_a = matrix_normalized[id_a,:]
        weight_b = matrix_normalized[id_b,:]
        sim = tf.tensordot(weight_b, weight_a,axes = 1).numpy()
        if sim < min_value:
            min_value = sim
            min_tuple = (numA,numB)
print('------')
print(min_value)
print(min_tuple)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [7]:
def find_nearest_k(searched_word,k):
    list = []
    id = id_dict[searched_word]
    searched_vector = matrix_normalized[id,:] 
    
    for word in id_dict:
        word_weights = matrix_normalized[id_dict[word]]
        loss = tf.tensordot(word_weights,searched_vector,axes = 1).numpy()
        list = insert(list,(word,loss))
        if len(list) > k:
            list = list[0:k+1]
    return list[0:k]

# Function to insert element
def insert(list, tuple):
    (word,n) = tuple
    if(len(list) == 0):
        list = [(word,n)]
    # Searching for the position
    for i in range(len(list)):
        (word_i,n_i) = list[i]
        if n_i < n:
            index = i
            break
      
    # Inserting n in the list
    list = list[:i] + [(word,n)] + list[i:]
    return list


print(find_nearest_k('batman',6))#this should produce one, for the same vectors
print(find_nearest_k('hogwarts',6))
print(find_nearest_k('turing',6))
print(find_nearest_k('florida',6))
print(find_nearest_k('object-oriented',6))
print(find_nearest_k('dancing',6))

[('batman', 1.0), ('superman', 0.6533917266554318), ('catwoman', 0.5784364578373318), ('superhero', 0.5565441009681275), ('spider-man', 0.5218831109165304), ('joker', 0.5129296574374288)]
[('hogwarts', 0.9999999999999998), ('dumbledore', 0.549229863608), ('wizardry', 0.5188968261681793), ('hermione', 0.4557605986849196), ('voldemort', 0.45157509151676123), ('hagrid', 0.4479209740521845)]
[('turing', 0.9999999999999998), ('deterministic', 0.5119534735350421), ('computable', 0.465302480354785), ('gödel', 0.45792113926794675), ('nondeterministic', 0.450346003250189), ('probabilistic', 0.43031487450612743)]
[('florida', 1.0000000000000002), ('miami', 0.6665895319646075), ('fla.', 0.6643667347542), ('carolina', 0.6428763922567252), ('texas', 0.6192212509245346), ('tallahassee', 0.5966705264371971)]
[('object-oriented', 0.9999999999999996), ('smalltalk', 0.6220564039940726), ('c++', 0.5849095890800992), ('scripting', 0.5446553503096182), ('implementations', 0.5332775991605072), ('domain-spec